In [1]:
import nltk
from nltk import *
from collections import Counter
import math

In [2]:
def lettura(corpus):
    with open(corpus, "r", encoding = "utf-8") as infile:
        file_lettura = infile.read()
        return file_lettura[:100000]

def analisiCorpus(file):
    frasi = sent_tokenize(file)
    tokens = []
    for frase in frasi:
        tokens += word_tokenize(frase)
    pos_tagging = pos_tag(tokens)
    return frasi, tokens, pos_tagging

def pos_ngrammi(tokens_tag, n):
    pos_tag = [tag for token, tag in tokens_tag] #creo una lista di pos tag dai token associati 
    pos_ngrammi = list(ngrams(pos_tag, n)) #creo una lista di n-grammi di pos utilizzando la funzione ngrams
    frequenza_ngrammi = FreqDist(pos_ngrammi) #utilizzo il modulo FreqDist per calcola la frequenza degli n-grammi
    #ottengo i 10 n-grammi più frequenti di pos tag utilizzando il metodo most_common
    top_ngrammi = frequenza_ngrammi.most_common(10) 
    return top_ngrammi

def top_tags(tokens, pos):
    #creo una lista di token filtrando quelli che hanno il pos tag richiesto
    filtra_token = [token for token, pos_tag in tokens if pos_tag == pos]
    freq_token = Counter(filtra_token) #utilizzo la funzione Counter per contare le occorrenze di ogni token nella lista filtrata
    top_tokens = freq_token.most_common(20) #ottengo i 20 tokens più frequenti
    return top_tokens

def bigrammi_nomi_aggettivi(tokens_tag):
    #uso la funzione bigrams del modulo nltk per creare una lista di bigrammi di pos 
    bigrammi = list(bigrams([pos for token, pos in tokens_tag]))
    aggettivo_nome = [] #lista vuota che conterrà tutti i bigrammi aggettivo-nome
    for i in range(len(tokens_tag)-1):
        #seleziono tutti i bigrammi che contengono un aggettivo seguito da un nome
        if bigrammi[i][0] == "JJ" and bigrammi [i][1] == "NN":
            aggettivo_nome.append((tokens_tag[i][0], tokens_tag[i+1][0])) #aggiungo il bigramma alla lista
    freq_dist = FreqDist(aggettivo_nome)
    top_bigrammi = freq_dist.most_common(20)
    return top_bigrammi, aggettivo_nome

    
def prob_cond(tokens, bigrammi):
    frequenze_unigrammi = FreqDist(tokens)
    probabilità_condizionate = {} #dizionario vuoto che conterrà le probabilità condizionate
    conteggio_token = FreqDist(bigrammi)
    for x, y in conteggio_token.items():
        if frequenze_unigrammi[x[0]] > 1: #filtro solo parole non hapax al denominatore
            denominatore = frequenze_unigrammi[x[0]] #corrisponde alla sequenza del token che condiziona il bigramma
            numeratore = y #corrisponde alla frequenza del bigramma
            probabilità = numeratore/denominatore
            probabilità_condizionate[x] = probabilità #aggiungo la probabilità condizionata al dizionario
    #ordino il dizionario di probabilità in maniera decrescente
    top_prob = sorted(probabilità_condizionate.items(), key = lambda x: x[1], reverse = True)[:20]
    return top_prob

def mutual_information(tokens, bigrammi):
    PMI = {} #inizializzo il dizionario che conterrà la PMI per ogni bigramma 
    conteggio_token = FreqDist(tokens)
    conteggio_bigrammi = FreqDist(bigrammi)
    for x, y in conteggio_bigrammi.items():
        numeratore = y * len(tokens) #moltiplico la frequenza del bigramma per la lunghezza del corpus
        #moltiplico la probabilità del primo termine del bigramma per quella del secondo
        denominatore = (conteggio_token[x[0]]/len(tokens)) * (conteggio_token[x[1]]/len(tokens))
        calcolo_PMI = numeratore/denominatore
        PMI[x] = math.log2(calcolo_PMI) #applico il logaritmo in base 2 alla mia probabilità
    top_pmi = sorted(PMI.items(), key = lambda x: x[1], reverse = True)[:20]
    return top_pmi

def distribuzione_frasi_max(tokens, frasi):
    freq_token = FreqDist(tokens)
    frasi_filtrate = []
    #calcolo la frequenza dei token e seleziono quelli non hapax
    non_hapax = [token for token, freq in freq_token.items() if freq >= 2]
    #filtro le frasi in base alla loro lunghezza e alla presenza di token non hapax
    for frase in frasi:
        tokens_frase = word_tokenize(frase)
        if len(tokens_frase) >= 10 and len(tokens_frase) <= 20:
            frequenza_token = 0
            for token in tokens_frase:
                if token in non_hapax:
                    frequenza_token += 1
            if frequenza_token >= len(tokens_frase) // 2:
                frasi_filtrate.append(tokens_frase)
    #seleziono la frase con la valutazione media di frequenza dei token maggiore tra le frasi filtrate
    max_freq = 0
    frase_freq_max = ""
    for sentence in frasi_filtrate:
        somma_frequenza = 0
        for token in sentence:
            somma_frequenza += freq_token[token]
        freq_media = somma_frequenza / len(sentence)
        if freq_media > max_freq:
            max_freq = freq_media
            frase_freq_max = sentence
    #restituisco la frase selezionata (sotto forma di stringa) 
    return " ".join(frase_freq_max)

def distribuzione_frasi_min(tokens, frasi):
    freq_token = FreqDist(tokens)
    frasi_filtrate = []
    non_hapax = [token for token, freq in freq_token.items() if freq >= 2]
    for frase in frasi:
        tokens_frase = word_tokenize(frase)
        if len(tokens_frase) >= 10 and len(tokens_frase) <= 20:
            frequenza_token = 0
            for token in tokens_frase:
                if token in non_hapax:
                    frequenza_token += 1
            if frequenza_token >= len(tokens_frase) // 2:
                frasi_filtrate.append(tokens_frase)
    #seleziono la fraso con la valutazione media di frequenza dei token minore tra le frasi filtrate
    min_freq = 1000
    frase_freq_min = ""
    for sentence in frasi_filtrate:
        somma_frequenza = 0
        for token in sentence:
            somma_frequenza += freq_token[token]
        freq_media = somma_frequenza / len(sentence)
        if freq_media < min_freq:
            min_freq = freq_media
            frase_freq_min = sentence
    return " ".join(frase_freq_min)

def markov2(tokens, frasi):
    freq_token = FreqDist(tokens)
    frasi_filtrate = []
    non_hapax = [token for token, freq in freq_token.items() if freq >= 2]
    for frase in frasi:
        tokens_frase = word_tokenize(frase)
        if len(tokens_frase) >= 10 and len(tokens_frase) <= 20:
            frequenza_token = 0
            for token in tokens_frase:
                if token in non_hapax:
                    frequenza_token += 1
            if frequenza_token >= len(tokens_frase) // 2:
                frasi_filtrate.append(tokens_frase)
    #calcolo la frase tra quelle filtrate con la probabilità massima tramite la catena di Markov di ordine 2
    frase_max = ""
    prob_max = 0
    bigrammi_corpus = list(bigrams(tokens))
    trigrammi_corpus = list(trigrams(tokens))
    for sentence in frasi_filtrate:
        tokens_frase = sentence
        bigrammi = list(bigrams(sentence))
        trigrammi = list(trigrams(sentence))
        primo_token = bigrammi[0][0]
        prob_primoToken = freq_token[primo_token] / len(tokens)
        prob_primoBigramma = bigrammi_corpus.count(bigrammi[0])/freq_token[primo_token]
        probMarkov1 = prob_primoToken * prob_primoBigramma
        prob_trigrammi = 1
        for trigramma in trigrammi:
            prob_trigrammi *= trigrammi_corpus.count(trigramma)/bigrammi_corpus.count((trigramma[0], trigramma[1]))
            probMarkov2 = probMarkov1 * prob_trigrammi
            if probMarkov2 > prob_max:
                prob_max = probMarkov2
                frase_max = sentence
    return " ".join(frase_max), prob_max

def distribuzione_ner(tokens, tokens_tag):
    #utilizzando il metodo ne_chunk, effettuo una classificazione delle entità nominate presenti nel testo
    ner_tags = nltk.ne_chunk(tokens_tag)  
    ner_freq = Counter()
    #conto le sole entità nominate
    for nodo in ner_tags:
        if hasattr(nodo, 'label'):
            classe = nodo.label()
            entità = " ".join([token for token, pos in nodo.leaves()])
            ner_freq[(entità, classe)] += 1
    #creazione dei dizionari per organizzazioni, persone e GPE
    organizzazioni = {}
    persone = {}
    gpe = {}
    for parola, classe in ner_freq.keys():
        if classe == 'ORGANIZATION':
            organizzazioni[parola, classe] = ner_freq[parola, classe]
        elif classe == 'PERSON':
            persone[parola, classe] = ner_freq[parola, classe]
        elif classe == 'GPE':
            gpe[parola, classe] = ner_freq[parola, classe]
    #seleziono le prime 15 entità più frequenti per ogni dizionario
    top15_organizzazioni = sorted(organizzazioni.items(), key = lambda x: x[1], reverse = True)[:15]
    top15_persone = sorted(persone.items(), key = lambda x: x[1], reverse = True)[:15]
    top15_gpe = sorted(gpe.items(), key = lambda x: x[1], reverse = True)[:15]
    return top15_organizzazioni, top15_persone, top15_gpe


In [3]:
file1 = "analisi_mente.txt"

def main(corpus1):
    lettura_file1 = lettura(corpus1)
    frasi1, tokens1, tokens_tag1 = analisiCorpus(lettura_file1)
    
     #ORDINO PER FREQUENZA DECRESCENTE I 10 UNIGRAMMI, BIGRAMMI E TRIGRAMMI DI POS PIU' FREQUENTI

    indici = [1, 2, 3]
    unigrammi1 = pos_ngrammi(tokens_tag1, indici[0])
    print(f'I dieci unigrammi sono: {unigrammi1}')
    bigrammi1 = pos_ngrammi(tokens_tag1, indici[1])
    print(f'I dieci bigrammi più frequenti sono: {bigrammi1}')
    trigrammi1 = pos_ngrammi(tokens_tag1, indici[2])
    print(f'I dieci trigrammi più frequenti sono {trigrammi1}')
    
    print()
    
    #ORDINO PER FREQUENZA DECRESCENTI I 10 NOMI, AGGETTIVI E VERBI PIU' FREQUENTI
    indici_tag = ['NN', 'JJ', 'VBD']
    top_nomi = top_tags(tokens_tag1, indici_tag[0])
    print(f'I 20 nomi più presenti nel testo {file1} sono: {top_nomi}')
    top_aggettivi = top_tags(tokens_tag1, indici_tag[1])
    print(f'I 20 aggettivi più presenti nel testo {file1} sono: {top_aggettivi}')
    top_verbi = top_tags(tokens_tag1, indici_tag[2])
    print(f'I 20 verbi più presenti nel testo {file1} sono: {top_verbi}')
    
    #ESTRAGGO I 20 BIGRAMMI COMPOSTI DA AGGETTIVO-SOSTANTIVO PIU' FREQUENTI
    
    print()
    top_bigrammi1, bigrammi1 = bigrammi_nomi_aggettivi(tokens_tag1)
    print(f'I venti bigrammi aggettivo-sostantivo più frequenti nel file {file1}: {top_bigrammi1}')
    
    #ESTRAGGO I 20 BIGRAMMI AGGETTIVO-SOSTANTIVO CON PROBABILITA' CONDIZIONATA MASSIMA
    
    prob_cond_bigrams1 = prob_cond(tokens1, bigrammi1)
    print()
    print(f'Venti bigrammi aggettivo-sostantivo con la probabilità condizionata massima nel file {file1}: {prob_cond_bigrams1}')
    
    #ESTRAGGO I 20 BIGRAMMI AGGETTIVO-SOSTANTIVO CON PMI MASSIMA
    top_pmi1 = mutual_information(tokens1, bigrammi1)
    print()
    print(f'Venti bigrammi aggettivo-sostantivo con forza associata massima nel file {file1}: {top_pmi1}')
    
    # DATA UNA SERIE DI CARATTERISTICHE, ESTRAGGO PER IL MIO CORPUS LA FRASE CON LA MEDIA DELLA DISTRIBUZIONE DI FREQUENZA MAGGIORE 
    # E FREQUENZA MINORE PER I TOKEN, OLTRE A QUELLA CON LA PROBABILITA' PIU' ALTA CON UN MODELLO DI MARKOV DI ORDINE 2
    
    frase_max1 = distribuzione_frasi_max(tokens1, frasi1)
    print()
    print(f'La frase con la media della distribuzione maggiore per il file {file1} è: {frase_max1}')
    print()
    frase_min1 = distribuzione_frasi_min(tokens1, frasi1)
    print(f'La frase con la media della distribuzione minore per il file {file1} è: {frase_min1}')
    print()
    print()
    prob_markov1, prob1 = markov2(tokens1, frasi1)
    print(f'La frase con la probabilità più alta secondo un modello di markov di ordine 2 nel file {file1} è: {prob_markov1} ({prob1})')
    
    #TOP 15 ENTITA' PIU' PRESENTI PER OGNI CLASSE 
    
    top15_organizzazioni1, top15_persone1, top15_gpe1 = distribuzione_ner(tokens1, tokens_tag1)
    print()
    print(f'I 15 nomi di organizzazioni più frequenti in {file1} sono: {top15_organizzazioni1}\n\nI 15 nomi di persona più frequenti in {file1} sono: {top15_persone1}\n\nI 15 nomi di entità geopolitiche più presenti in {file1} sono {top15_gpe1}')
    
main(file1)

I dieci unigrammi sono: [(('IN',), 2560), (('NN',), 2506), (('DT',), 1895), (('JJ',), 1465), ((',',), 1177), (('PRP',), 945), (('RB',), 925), (('NNS',), 923), (('VBZ',), 827), (('VB',), 748)]
I dieci bigrammi più frequenti sono: [(('DT', 'NN'), 1009), (('IN', 'DT'), 871), (('NN', 'IN'), 801), (('JJ', 'NN'), 526), (('DT', 'JJ'), 430), (('NN', ','), 427), (('IN', 'NN'), 391), (('PRP', 'VBP'), 325), (('TO', 'VB'), 309), (('JJ', 'NNS'), 308)]
I dieci trigrammi più frequenti sono [(('IN', 'DT', 'NN'), 480), (('DT', 'NN', 'IN'), 438), (('DT', 'JJ', 'NN'), 312), (('NN', 'IN', 'DT'), 245), (('NN', 'IN', 'NN'), 195), (('IN', 'DT', 'JJ'), 192), (('JJ', 'NN', 'IN'), 164), (('PRP', 'MD', 'VB'), 136), (('DT', 'NN', ','), 117), (('JJ', 'NN', ','), 116)]

I 20 nomi più presenti nel testo analisi_mente.txt sono: [('desire', 52), ('matter', 47), ('mind', 42), ('behaviour', 39), ('consciousness', 37), ('view', 37), ('animal', 37), ('thought', 36), ('object', 33), ('something', 27), ('observation', 27), 